### Looking at the Overlap between single id's in Dimensions and OpenAlex

In [20]:
import csv
import pandas as pd
import numpy as np
import json 
import requests
from nameparser import HumanName


In [12]:
with open("nameOneIDs.csv", encoding="latin-1") as fileToRead:
    csvReader = csv.DictReader(fileToRead)   # create reader object
    rows = [row for row in csvReader]        # do the reading of each row

print(len(rows))
df = pd.DataFrame(data=rows)
names = df['Name']
#print(names)

993


In [25]:

# Initialize a dictionary to store the queried IDs for each physician-scientist
ids_dictionary = {}

# Initialize a set to store the failed queries
failed_queries = set()
count = 0

for person in names:
    cursor = "*"
    author_name = person.lower()
    # Keep making queries until the request is successful or the maximum number of queries is reached
    while True:
        query_author = f'https://api.openalex.org/authors?search={author_name}&per_page=100&cursor={cursor}'
        
        try:
            # Make the API request
            response = requests.get(query_author)


            # Check if the request was successful
            if response.status_code == 200:
                # Get the list of inventors from the response
                authors = response.json()["results"]

                # Iterate over the author and add them to the dictionary
                for author in authors:
                    name = HumanName(author.pop("display_name"))
    
                    # remove unnecessary features
                    author.pop("display_name_alternatives")
                    author.pop("orcid")
                    author.pop("summary_stats")
                    author.pop("x_concepts")
                    author.pop("works_api_url")
                    
                    author["id"] = author["id"][21:]
                    author["name_first"] = name.first if len(name.first) > 1 else None
                    author["name_middle"] = name.middle if len(name.middle) > 1 else None
                    author["name_last"] = name.last if len(name.last) > 1 else None
                    author["works"] = []
                    
                    author["api"] = "openAlex"
                    
                    if author_name not in ids_dictionary:
                        ids_dictionary[author_name] = {}
                    
                    if author["id"] not in ids_dictionary[author_name]:
                        ids_dictionary[author_name][author["id"]] = {}

                    ids_dictionary[author_name][author["id"]] = author

            # Break out of the loop if there are no more authors
            if not authors:
                break

            # Update the query parameters with the next `cursor` value
            cursor = response.json()["meta"]["next_cursor"]
                
        # If the request is unsuccessful, add the query name and `after` value to the failed queries set
        except Exception as e:
            print(e)
            failed_queries.add((author_name, cursor))
            break
    count +=1
    print(f'{count}. {author_name} + done')

# Print the failed queries
print(f"Failed queries: {failed_queries}")

1. yale nemerson + done
2. elia ayoub + done
3. john dietschy + done
4. yale enson + done
5. mehran goulian + done
6. norman javitt + done
7. hermes kontos + done
8. gilbert levinson + done
9. harvey marver + done
10. harold neu + done
11. arthur eisen + done
12. barry fanburg + done
13. dorothea zucker-franklin + done
14. nathan zvaifler + done
15. phyllis bodel + done
16. sidney cooperband + done
17. jacques kessler + done
18. stanley korenman + done
19. gildon beall + done
20. neil cherniack + done
21. blas frangione + done
22. irwin freedberg + done
23. thomas merimee + done
24. i samloff + done
25. wadi suki + done
26. shirley ebbe + done
27. clyde bardin + done
28. ward bullock + done
29. jack coburn + done
30. richard levere + done
31. michael mosesson + done
32. ursula muller-eberhard + done
33. leon sabath + done
34. burton sobel + done
35. martin surks + done
36. siamak adibi + done
37. carl bentzel + done
38. kevin catt + done
39. wallace clyde + done
40. harvey colten + don

In [27]:
with open("openAlex_ids.json", "w") as f:
    json.dump(ids_dictionary, f, indent=4)

# with open("openAlex_failed_queries.json", "w") as f:
#     json.dump(list(failed_queries), f, indent=4)

In [34]:
single_OP = []
with open("openAlex_ids.json", "r") as f:
    ids_dictionary = json.load(f)
for name in ids_dictionary.keys():
    if len(ids_dictionary[name]) == 1:
        single_OP.append(name)
print(single_OP)
print(len(single_OP))


['yale nemerson', 'elia ayoub', 'john dietschy', 'yale enson', 'mehran goulian', 'hermes kontos', 'gilbert levinson', 'harvey marver', 'dorothea zucker-franklin', 'nathan zvaifler', 'phyllis bodel', 'sidney cooperband', 'jacques kessler', 'gildon beall', 'neil cherniack', 'shirley ebbe', 'clyde bardin', 'michael mosesson', 'leon sabath', 'jamshid javid', 'marshall lichtman', 'manning thaler', 'harry beaty', 'viktor bokisch', 'alexander fefer', 'darryl granner', 'sheldon rothenberg', 'edgar cathcart', 'franklyn knox', 'julian niemetz', 'antero so', 'leonard jarett', 'edwin beachey', 'guenther boden', 'vincent discala', 'laurence kedes', 'robert adelstein', 'joseph avruch', 'frederick derubertis', 'jack hawiger', 'ruy soeiro', 'bobby stinebaugh', 'kirk wuepper', 'philip coffino', 'daniel roncari', 'irwin scher', 'nicholas kredich', 'makio ogawa', 'matthew rechler', 'david rosenstreich', 'shigeru sassa', 'neil blacklow', 'philip guzelian', 'ione kourides', 'harry margolius', 'richard shad

In [39]:
Open_lower = [name.lower() for name in single_OP]
Dimens_lower = [name.lower() for name in names]

# Find the intersection of lowercase names
intersection_result = set(Open_lower).intersection(Dimens_lower)
intersection_list = list(intersection_result)

print(len(intersection_list))

#check
checker = False
for name in Open_lower:
    if name in Dimens_lower:
        checker = True
print(checker)


330
True


In [40]:
with open("combined_singleIds.txt", "w") as file:
    for name in intersection_list:
        file.write(name + '\n')